In [1]:
import torch
from torch import nn
import numpy as np
import pandas

In [2]:
features_path = "features.csv"
labels_path = "labels.csv"
dataset_path = "data.csv"
new_dataset = "output_frame.csv"

In [3]:
class Network(nn.Module):
    def __init__(self):
        super().__init__()

        # Inputs to hidden layer linear transformation
        self.input = nn.Linear(6, 5)
        self.first_hidden = nn.Linear(5, 5)
        self.second_hidden = nn.Linear(5, 5)
        self.third_hidden = nn.Linear(5, 5)
        self.output = nn.Linear(5, 1)

        self.relu = nn.ReLU()

    def forward(self, x:[])->[]:
        # 3 -> 2
        x = self.input(x)
        x = self.relu(x)
        # 5 -> 5
        x = self.first_hidden(x)
        x = self.relu(x)
        # 5 -> 5
        x = self.second_hidden(x)
        x = self.relu(x)
        # 5 -> 5
        x = self.third_hidden(x)
        x = self.relu(x)
        # 2 -> 2
        x = self.output(x)

        return x

In [4]:
def get_dataset(path_to_dataset:str)->pandas.DataFrame:
    return pandas.read_csv(path_to_dataset, header=None)


def get_dataset_no_zeroes(dataframe:pandas.DataFrame)->pandas.DataFrame:
    return dataframe[dataframe[0] != 0]


def get_features(dataframe:pandas.DataFrame)->torch.Tensor:
    return dataframe[["Fill Factor", "Pitch", "Duty Cycle", "Theta", "Lambda", "Mode"]]


def get_labels(dataframe:pandas.DataFrame)->torch.Tensor:
    return dataframe[['Transmission']]


def get_features_no_zeroes(dataframe:pandas.DataFrame)->torch.Tensor:
    return torch.tensor(dataframe[dataframe[0] != 0].iloc[:,[0,1,2]].values)


def get_labels_no_zeroes(dataframe:pandas.DataFrame)->torch.Tensor:
    return torch.tensor(dataframe[dataframe[0] != 0].iloc[:,[3, 4]].values)
    

def transform_features(DataFrame:pandas.DataFrame)->torch.tensor:
    tensor = torch.tensor(DataFrame.values)
    tensor[:,0:5] = np.log10(tensor[:,0:5])
    return tensor


def transform_labels(DataFrame:pandas.DataFrame)->torch.tensor:
    tensor = torch.tensor(DataFrame.values)
    tensor[:,0] = np.log10(np.abs(tensor[:,0]))
    return tensor


def norm(tensor:torch.tensor)->torch.tensor:
    return torch.nn.functional.normalize(tensor).float()
    
    
def magnitude(vector:np.array)->float:
    return np.linalg.norm(vector)
    

In [5]:
dataset = pandas.read_csv("DATA_FILES/dataset.csv")
dataset["Mode"] = dataset["Mode"].astype('category').cat.codes
dataset = dataset[dataset["Mode"]!=0]
dataset

,Fill Factor,Pitch,Duty Cycle,Theta,Lambda,Transmission,Mode
0,0.2,5.000000e-07,0.4,5,0.000002,0.000097,1
1,0.2,5.000000e-07,0.4,5,0.000002,0.000098,1
2,0.2,5.000000e-07,0.4,5,0.000002,0.000100,1
3,0.2,5.000000e-07,0.4,5,0.000002,0.000102,1
4,0.2,5.000000e-07,0.4,5,0.000002,0.000104,1
...,...,...,...,...,...,...,...
518195,0.6,9.000000e-07,0.8,20,0.000001,0.000372,1
518196,0.6,9.000000e-07,0.8,20,0.000001,0.000339,1
518197,0.6,9.000000e-07,0.8,20,0.000001,0.000337,1
518198,0.6,9.000000e-07,0.8,20,0.000001,0.000354,1


In [22]:
# GET THE TRAINING SET
training_set = dataset.sample(frac = 0.85)
training_set

,Fill Factor,Pitch,Duty Cycle,Theta,Lambda,Transmission,Mode
104973,0.52,7.000000e-07,0.48,8,0.000001,0.001587,1
366831,0.60,7.000000e-07,0.56,17,0.000002,0.006288,1
399291,0.60,9.000000e-07,0.80,17,0.000001,0.000515,1
430055,0.60,9.000000e-07,0.80,17,0.000002,0.000470,1
300892,0.36,9.000000e-07,0.80,14,0.000001,0.001175,1
...,...,...,...,...,...,...,...
418983,0.60,9.000000e-07,0.80,17,0.000001,0.005123,1
3222,0.36,5.000000e-07,0.48,5,0.000002,0.000215,1
462412,0.52,9.000000e-07,0.40,20,0.000002,0.000357,1
509237,0.60,9.000000e-07,0.80,20,0.000002,0.003693,1


In [23]:
training_set.to_csv('training_set.csv')

In [24]:
testing_set = dataset.drop(training_set.index)
testing_set

,Fill Factor,Pitch,Duty Cycle,Theta,Lambda,Transmission,Mode
10,0.2,5.000000e-07,0.4,5,0.000002,0.000086,1
26,0.2,5.000000e-07,0.4,5,0.000002,0.000153,1
32,0.2,5.000000e-07,0.4,5,0.000002,0.000317,1
36,0.2,5.000000e-07,0.4,5,0.000002,0.000318,1
43,0.2,5.000000e-07,0.4,5,0.000002,0.000181,1
...,...,...,...,...,...,...,...
518157,0.6,9.000000e-07,0.8,20,0.000001,0.003485,1
518162,0.6,9.000000e-07,0.8,20,0.000001,0.007020,1
518163,0.6,9.000000e-07,0.8,20,0.000001,0.007828,1
518167,0.6,9.000000e-07,0.8,20,0.000001,0.009431,1


In [25]:
testing_set.to_csv('testing_set.csv')

In [27]:
GratingCouplerNet = torch.load("GratingCouplerNetModel_new")
GratingCouplerNet.eval()

Network(
  (input): Linear(in_features=6, out_features=100, bias=True)
  (first_hidden): Linear(in_features=100, out_features=100, bias=True)
  (second_hidden): Linear(in_features=100, out_features=50, bias=True)
  (third_hidden): Linear(in_features=50, out_features=50, bias=True)
  (output): Linear(in_features=50, out_features=1, bias=True)
  (relu): ReLU()
)

In [12]:
prediction = GratingCouplerNet(norm(transform_features(get_features(testing_set))))
prediction

tensor([[0.1155],
        [0.1155],
        [0.1155],
        ...,
        [0.1159],
        [0.1159],
        [0.1159]], grad_fn=<AddmmBackward>)

In [13]:
for value in prediction:
    print(value)

tensor([0.1155], grad_fn=<UnbindBackward>)
tensor([0.1155], grad_fn=<UnbindBackward>)
tensor([0.1155], grad_fn=<UnbindBackward>)
tensor([0.1155], grad_fn=<UnbindBackward>)
tensor([0.1154], grad_fn=<UnbindBackward>)
tensor([0.1154], grad_fn=<UnbindBackward>)
tensor([0.1154], grad_fn=<UnbindBackward>)
tensor([0.1154], grad_fn=<UnbindBackward>)
tensor([0.1154], grad_fn=<UnbindBackward>)
tensor([0.1154], grad_fn=<UnbindBackward>)
tensor([0.1154], grad_fn=<UnbindBackward>)
tensor([0.1154], grad_fn=<UnbindBackward>)
tensor([0.1154], grad_fn=<UnbindBackward>)
tensor([0.1154], grad_fn=<UnbindBackward>)
tensor([0.1154], grad_fn=<UnbindBackward>)
tensor([0.1154], grad_fn=<UnbindBackward>)
tensor([0.1154], grad_fn=<UnbindBackward>)
tensor([0.1154], grad_fn=<UnbindBackward>)
tensor([0.1154], grad_fn=<UnbindBackward>)
tensor([0.1154], grad_fn=<UnbindBackward>)
tensor([0.1154], grad_fn=<UnbindBackward>)
tensor([0.1154], grad_fn=<UnbindBackward>)
tensor([0.1154], grad_fn=<UnbindBackward>)
tensor([0.1

tensor([0.1164], grad_fn=<UnbindBackward>)
tensor([0.1164], grad_fn=<UnbindBackward>)
tensor([0.1164], grad_fn=<UnbindBackward>)
tensor([0.1164], grad_fn=<UnbindBackward>)
tensor([0.1164], grad_fn=<UnbindBackward>)
tensor([0.1164], grad_fn=<UnbindBackward>)
tensor([0.1164], grad_fn=<UnbindBackward>)
tensor([0.1164], grad_fn=<UnbindBackward>)
tensor([0.1164], grad_fn=<UnbindBackward>)
tensor([0.1150], grad_fn=<UnbindBackward>)
tensor([0.1150], grad_fn=<UnbindBackward>)
tensor([0.1150], grad_fn=<UnbindBackward>)
tensor([0.1150], grad_fn=<UnbindBackward>)
tensor([0.1150], grad_fn=<UnbindBackward>)
tensor([0.1149], grad_fn=<UnbindBackward>)
tensor([0.1149], grad_fn=<UnbindBackward>)
tensor([0.1149], grad_fn=<UnbindBackward>)
tensor([0.1149], grad_fn=<UnbindBackward>)
tensor([0.1149], grad_fn=<UnbindBackward>)
tensor([0.1149], grad_fn=<UnbindBackward>)
tensor([0.1149], grad_fn=<UnbindBackward>)
tensor([0.1149], grad_fn=<UnbindBackward>)
tensor([0.1149], grad_fn=<UnbindBackward>)
tensor([0.1

tensor([0.1152], grad_fn=<UnbindBackward>)
tensor([0.1152], grad_fn=<UnbindBackward>)
tensor([0.1152], grad_fn=<UnbindBackward>)
tensor([0.1152], grad_fn=<UnbindBackward>)
tensor([0.1152], grad_fn=<UnbindBackward>)
tensor([0.1152], grad_fn=<UnbindBackward>)
tensor([0.1152], grad_fn=<UnbindBackward>)
tensor([0.1152], grad_fn=<UnbindBackward>)
tensor([0.1152], grad_fn=<UnbindBackward>)
tensor([0.1152], grad_fn=<UnbindBackward>)
tensor([0.1152], grad_fn=<UnbindBackward>)
tensor([0.1152], grad_fn=<UnbindBackward>)
tensor([0.1151], grad_fn=<UnbindBackward>)
tensor([0.1151], grad_fn=<UnbindBackward>)
tensor([0.1151], grad_fn=<UnbindBackward>)
tensor([0.1151], grad_fn=<UnbindBackward>)
tensor([0.1151], grad_fn=<UnbindBackward>)
tensor([0.1151], grad_fn=<UnbindBackward>)
tensor([0.1151], grad_fn=<UnbindBackward>)
tensor([0.1151], grad_fn=<UnbindBackward>)
tensor([0.1151], grad_fn=<UnbindBackward>)
tensor([0.1151], grad_fn=<UnbindBackward>)
tensor([0.1151], grad_fn=<UnbindBackward>)
tensor([0.1

tensor([0.1147], grad_fn=<UnbindBackward>)
tensor([0.1147], grad_fn=<UnbindBackward>)
tensor([0.1147], grad_fn=<UnbindBackward>)
tensor([0.1147], grad_fn=<UnbindBackward>)
tensor([0.1147], grad_fn=<UnbindBackward>)
tensor([0.1147], grad_fn=<UnbindBackward>)
tensor([0.1147], grad_fn=<UnbindBackward>)
tensor([0.1146], grad_fn=<UnbindBackward>)
tensor([0.1146], grad_fn=<UnbindBackward>)
tensor([0.1146], grad_fn=<UnbindBackward>)
tensor([0.1146], grad_fn=<UnbindBackward>)
tensor([0.1146], grad_fn=<UnbindBackward>)
tensor([0.1152], grad_fn=<UnbindBackward>)
tensor([0.1152], grad_fn=<UnbindBackward>)
tensor([0.1152], grad_fn=<UnbindBackward>)
tensor([0.1152], grad_fn=<UnbindBackward>)
tensor([0.1152], grad_fn=<UnbindBackward>)
tensor([0.1152], grad_fn=<UnbindBackward>)
tensor([0.1152], grad_fn=<UnbindBackward>)
tensor([0.1152], grad_fn=<UnbindBackward>)
tensor([0.1152], grad_fn=<UnbindBackward>)
tensor([0.1151], grad_fn=<UnbindBackward>)
tensor([0.1151], grad_fn=<UnbindBackward>)
tensor([0.1

tensor([0.1155], grad_fn=<UnbindBackward>)
tensor([0.1155], grad_fn=<UnbindBackward>)
tensor([0.1155], grad_fn=<UnbindBackward>)
tensor([0.1155], grad_fn=<UnbindBackward>)
tensor([0.1155], grad_fn=<UnbindBackward>)
tensor([0.1155], grad_fn=<UnbindBackward>)
tensor([0.1155], grad_fn=<UnbindBackward>)
tensor([0.1155], grad_fn=<UnbindBackward>)
tensor([0.1155], grad_fn=<UnbindBackward>)
tensor([0.1155], grad_fn=<UnbindBackward>)
tensor([0.1155], grad_fn=<UnbindBackward>)
tensor([0.1155], grad_fn=<UnbindBackward>)
tensor([0.1154], grad_fn=<UnbindBackward>)
tensor([0.1154], grad_fn=<UnbindBackward>)
tensor([0.1154], grad_fn=<UnbindBackward>)
tensor([0.1154], grad_fn=<UnbindBackward>)
tensor([0.1154], grad_fn=<UnbindBackward>)
tensor([0.1154], grad_fn=<UnbindBackward>)
tensor([0.1154], grad_fn=<UnbindBackward>)
tensor([0.1154], grad_fn=<UnbindBackward>)
tensor([0.1154], grad_fn=<UnbindBackward>)
tensor([0.1154], grad_fn=<UnbindBackward>)
tensor([0.1154], grad_fn=<UnbindBackward>)
tensor([0.1

...

tensor([0.1160], grad_fn=<UnbindBackward>)
tensor([0.1160], grad_fn=<UnbindBackward>)
tensor([0.1159], grad_fn=<UnbindBackward>)
tensor([0.1159], grad_fn=<UnbindBackward>)
tensor([0.1159], grad_fn=<UnbindBackward>)
tensor([0.1159], grad_fn=<UnbindBackward>)
tensor([0.1159], grad_fn=<UnbindBackward>)
tensor([0.1159], grad_fn=<UnbindBackward>)
tensor([0.1159], grad_fn=<UnbindBackward>)
tensor([0.1159], grad_fn=<UnbindBackward>)
tensor([0.1159], grad_fn=<UnbindBackward>)
tensor([0.1161], grad_fn=<UnbindBackward>)
tensor([0.1161], grad_fn=<UnbindBackward>)
tensor([0.1161], grad_fn=<UnbindBackward>)
tensor([0.1161], grad_fn=<UnbindBackward>)
tensor([0.1161], grad_fn=<UnbindBackward>)
tensor([0.1161], grad_fn=<UnbindBackward>)
tensor([0.1161], grad_fn=<UnbindBackward>)
tensor([0.1160], grad_fn=<UnbindBackward>)
tensor([0.1160], grad_fn=<UnbindBackward>)
tensor([0.1160], grad_fn=<UnbindBackward>)
tensor([0.1160], grad_fn=<UnbindBackward>)
tensor([0.1160], grad_fn=<UnbindBackward>)
tensor([0.1

tensor([0.1159], grad_fn=<UnbindBackward>)
tensor([0.1161], grad_fn=<UnbindBackward>)
tensor([0.1161], grad_fn=<UnbindBackward>)
tensor([0.1161], grad_fn=<UnbindBackward>)
tensor([0.1161], grad_fn=<UnbindBackward>)
tensor([0.1161], grad_fn=<UnbindBackward>)
tensor([0.1160], grad_fn=<UnbindBackward>)
tensor([0.1160], grad_fn=<UnbindBackward>)
tensor([0.1160], grad_fn=<UnbindBackward>)
tensor([0.1160], grad_fn=<UnbindBackward>)
tensor([0.1160], grad_fn=<UnbindBackward>)
tensor([0.1160], grad_fn=<UnbindBackward>)
tensor([0.1160], grad_fn=<UnbindBackward>)
tensor([0.1160], grad_fn=<UnbindBackward>)
tensor([0.1160], grad_fn=<UnbindBackward>)
tensor([0.1160], grad_fn=<UnbindBackward>)
tensor([0.1160], grad_fn=<UnbindBackward>)
tensor([0.1160], grad_fn=<UnbindBackward>)
tensor([0.1160], grad_fn=<UnbindBackward>)
tensor([0.1160], grad_fn=<UnbindBackward>)
tensor([0.1160], grad_fn=<UnbindBackward>)
tensor([0.1160], grad_fn=<UnbindBackward>)
tensor([0.1160], grad_fn=<UnbindBackward>)
tensor([0.1

tensor([0.1161], grad_fn=<UnbindBackward>)
tensor([0.1161], grad_fn=<UnbindBackward>)
tensor([0.1161], grad_fn=<UnbindBackward>)
tensor([0.1160], grad_fn=<UnbindBackward>)
tensor([0.1160], grad_fn=<UnbindBackward>)
tensor([0.1160], grad_fn=<UnbindBackward>)
tensor([0.1160], grad_fn=<UnbindBackward>)
tensor([0.1160], grad_fn=<UnbindBackward>)
tensor([0.1160], grad_fn=<UnbindBackward>)
tensor([0.1160], grad_fn=<UnbindBackward>)
tensor([0.1160], grad_fn=<UnbindBackward>)
tensor([0.1160], grad_fn=<UnbindBackward>)
tensor([0.1160], grad_fn=<UnbindBackward>)
tensor([0.1160], grad_fn=<UnbindBackward>)
tensor([0.1160], grad_fn=<UnbindBackward>)
tensor([0.1160], grad_fn=<UnbindBackward>)
tensor([0.1160], grad_fn=<UnbindBackward>)
tensor([0.1160], grad_fn=<UnbindBackward>)
tensor([0.1160], grad_fn=<UnbindBackward>)
tensor([0.1160], grad_fn=<UnbindBackward>)
tensor([0.1160], grad_fn=<UnbindBackward>)
tensor([0.1160], grad_fn=<UnbindBackward>)
tensor([0.1160], grad_fn=<UnbindBackward>)
tensor([0.1

tensor([0.1160], grad_fn=<UnbindBackward>)
tensor([0.1159], grad_fn=<UnbindBackward>)
tensor([0.1159], grad_fn=<UnbindBackward>)
tensor([0.1159], grad_fn=<UnbindBackward>)
tensor([0.1159], grad_fn=<UnbindBackward>)
tensor([0.1159], grad_fn=<UnbindBackward>)
tensor([0.1159], grad_fn=<UnbindBackward>)
tensor([0.1161], grad_fn=<UnbindBackward>)
tensor([0.1161], grad_fn=<UnbindBackward>)
tensor([0.1161], grad_fn=<UnbindBackward>)
tensor([0.1161], grad_fn=<UnbindBackward>)
tensor([0.1161], grad_fn=<UnbindBackward>)
tensor([0.1161], grad_fn=<UnbindBackward>)
tensor([0.1161], grad_fn=<UnbindBackward>)
tensor([0.1161], grad_fn=<UnbindBackward>)
tensor([0.1160], grad_fn=<UnbindBackward>)
tensor([0.1160], grad_fn=<UnbindBackward>)
tensor([0.1160], grad_fn=<UnbindBackward>)
tensor([0.1160], grad_fn=<UnbindBackward>)
tensor([0.1160], grad_fn=<UnbindBackward>)
tensor([0.1160], grad_fn=<UnbindBackward>)
tensor([0.1160], grad_fn=<UnbindBackward>)
tensor([0.1160], grad_fn=<UnbindBackward>)
tensor([0.1

tensor([0.1159], grad_fn=<UnbindBackward>)
tensor([0.1159], grad_fn=<UnbindBackward>)
tensor([0.1159], grad_fn=<UnbindBackward>)
tensor([0.1159], grad_fn=<UnbindBackward>)
tensor([0.1159], grad_fn=<UnbindBackward>)
tensor([0.1161], grad_fn=<UnbindBackward>)
tensor([0.1161], grad_fn=<UnbindBackward>)
tensor([0.1161], grad_fn=<UnbindBackward>)
tensor([0.1161], grad_fn=<UnbindBackward>)
tensor([0.1160], grad_fn=<UnbindBackward>)
tensor([0.1160], grad_fn=<UnbindBackward>)
tensor([0.1160], grad_fn=<UnbindBackward>)
tensor([0.1160], grad_fn=<UnbindBackward>)
tensor([0.1160], grad_fn=<UnbindBackward>)
tensor([0.1160], grad_fn=<UnbindBackward>)
tensor([0.1160], grad_fn=<UnbindBackward>)
tensor([0.1160], grad_fn=<UnbindBackward>)
tensor([0.1160], grad_fn=<UnbindBackward>)
tensor([0.1160], grad_fn=<UnbindBackward>)
tensor([0.1160], grad_fn=<UnbindBackward>)
tensor([0.1159], grad_fn=<UnbindBackward>)
tensor([0.1159], grad_fn=<UnbindBackward>)
tensor([0.1159], grad_fn=<UnbindBackward>)
tensor([0.1

In [14]:
transform_labels(get_labels(training_set))

tensor([[-3.3902],
        [-3.8720],
        [-3.4289],
        ...,
        [-2.7130],
        [-4.8827],
        [-4.8097]], dtype=torch.float64)

In [29]:
np.power(10, prediction[0].detach().numpy())

array([0.00022], dtype=float32)

In [30]:
np.power(10, y_test[0].detach().numpy())

array([0.00010179])

In [648]:
-1*np.power(10, y_test[0][1].detach().numpy())

-0.01945726046799541

In [651]:
-1*np.power(10, gratingCouplerNet(X_test_normed[0]).detach().numpy()[1])

-0.005269782584722879

In [654]:
-1*np.power(10, y[0][1].detach().numpy())

-0.005151365999223099

In [655]:
-1*np.power(10, gratingCouplerNet(X_normed[0]).detach().numpy()[1])

-0.008357428100215211

In [728]:
X_new = transform_features(Dataset.get_features(new_dataset))
X_new_normed = norm(X_new)
y_new = transform_labels(Dataset.get_labels(new_dataset))

In [731]:
new_prediction = gratingCouplerNet(X_new_normed)

In [732]:
y_new[0]

tensor([-5.7959, -0.1303])

In [730]:
new_prediction[0]

tensor([-8.8410, -8.5823], grad_fn=<SelectBackward>)

In [734]:
loss_function(new_prediction, y_new.float())

tensor(1.9358, grad_fn=<MseLossBackward>)